In [45]:
import os, glob
import torch
import pandas as pd
import numpy as np
import pickle
import joblib
import argparse

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

In [47]:
from train_RF import get_name, load, setup, sample, save, prepare_dataset, create_dataset

In [48]:
import sys
sys.argv = [
    'train_RF.py',                   
    '../../input_imgs',
    '../../JPEGAI_output/'         
    
]
# --- Setup an argument parser --- #
# Arguments for coder
parser = argparse.ArgumentParser(description='Compress a directory of images using the RecoEncoder')
parser.add_argument('--gpu', type=int, default=0, help='GPU index')
parser.add_argument('--imgs_path', type=str, default='../../real_vs_fake/real-vs-fake', help='Input directory')
parser.add_argument('input_path', type=str, default='../../input_imgs', help='Input directory')
parser.add_argument('bin_path', type=str, default='../../JPEGAI_output/', help='Save directory')
parser.add_argument('--set_target_bpp', type=int, default=1, help='Set the target bpp '
                                                                    '(multiplied by 100)')
parser.add_argument('--models_dir_name', type=str, default='../../jpeg-ai-reference-software/models', help='Directory name for the '
                                                                                'models used in the encoder-decoder'
                                                                                'pipeline')
#Arguments for training
parser.add_argument('--num_samples', type=int, default=1000, help='Number of samples to train on')
parser.add_argument('--num_samples_test', type=int, default=300, help='Number of samples to test on')
parser.add_argument('--random_sample', type=bool, default=False, help='Sample')
parser.add_argument("--train_csv",default="../../train.csv" , help="Path to dataset's csv file")
parser.add_argument("--test_csv", default="../../test.csv", help="Path to test's csv file")
parser.add_argument("-t", "--target",default=None, help="y_hat if quantized latent, else y")
parser.add_argument("--save", default=False, help="True if wanted to save dataset")
parser.add_argument("--models_save_dir", default="/data/lesc/users/rustichini/thesis/models_saved", help="Directory to save models")

args = parser.parse_args()

#train_process(args)

In [49]:
imgs_dir = args.imgs_path

df_test = pd.read_csv(args.test_csv)
if args.num_samples is not None:
    df_test = sample(df_test, args.num_samples)

num_test = df_test.shape[0] 

save_latent_path = os.path.join(args.bin_path,str(args.set_target_bpp)+"_bpp","test","latent")

In [50]:
print(args)

Namespace(bin_path='../../JPEGAI_output/', gpu=0, imgs_path='../../real_vs_fake/real-vs-fake', input_path='../../input_imgs', models_dir_name='../../jpeg-ai-reference-software/models', models_save_dir='/data/lesc/users/rustichini/thesis/models_saved', num_samples=1000, num_samples_test=300, random_sample=False, save=False, set_target_bpp=1, target=None, test_csv='../../test.csv', train_csv='../../train.csv')


In [52]:
coder = setup(args)
df_test = pd.read_csv(args.test_csv)

if df_test.shape[0] > args.num_samples_test:
    df_test = sample(df_test,args.num_samples_test)

save_latent_path = os.path.join(args.bin_path,str(args.set_target_bpp)+"_bpp","test","latent")

print(save_latent_path)
X_test, _, y_test = create_dataset(coder, df_test, args.imgs_path, save_latent_path)


Using GPU 0
Codec/Encoder: name=JAI, version=DIS
../../JPEGAI_output/1_bpp/test/latent
BEGIN
            label label_str                      path
id                                                   
18233           1      real       test/real/18233.jpg
54317           1      real       test/real/54317.jpg
40155           1      real       test/real/40155.jpg
12875           1      real       test/real/12875.jpg
52543           1      real       test/real/52543.jpg
...           ...       ...                       ...
7VVURTOD9Z      0      fake  test/fake/7VVURTOD9Z.jpg
CWEXDCDHA2      0      fake  test/fake/CWEXDCDHA2.jpg
BF8NSMWWC9      0      fake  test/fake/BF8NSMWWC9.jpg
B9NJCLL40W      0      fake  test/fake/B9NJCLL40W.jpg
VF6O429TGS      0      fake  test/fake/VF6O429TGS.jpg

[300 rows x 3 columns]


Creating dataset:   0%|                                                     | 0/300 [00:00<?, ?it/s]


NameError: name 'args' is not defined

In [ ]:
model_path = "/data/lesc/users/rustichini/thesis/models_saved/6_bpp/1000_samples/y/RF_final_200trees_0.1features.joblib"
model = load(model_path)
print(model)
model.verbose=0


RandomForestClassifier(max_depth=20, max_features=0.1, min_samples_leaf=5,
                       min_samples_split=20, n_estimators=200, n_jobs=8,
                       random_state=42, verbose=1)


In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
model_path = "/data/lesc/users/rustichini/thesis/models_saved/6_bpp/30000_samples/y_hat/grid_search.joblib"
model = load(model_path)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      1000
           1       0.77      0.82      0.79      1000

    accuracy                           0.79      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.79      0.79      2000

0.786


In [ ]:
model_path = "/data/lesc/users/rustichini/thesis/models_saved/6_bpp/25000_samples/y/RF_100_estimators.joblib"
model = load(model_path)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.04      0.08      1000
           1       0.51      0.98      0.67      1000

    accuracy                           0.51      2000
   macro avg       0.59      0.51      0.37      2000
weighted avg       0.59      0.51      0.37      2000

0.511
